In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## EWMA

### Exponential Moving Average
Simple Moving Average (SMA) time series are much less noisy than the time series of the original data points. The challenge with SMA, however, is that the values of SMA lag the original values. This means that changes in the trend are only seen with a delay (lag) of L time units. For datasets that contain rapid trend shifts, a SMA may simply be too slow to be useful. 

**Exponential Moving Average (EMA)** helps reduce the lag induced by the use of the SMA. It does this by putting more weight on more recent observations, while the SMA weights all observations equally.

The EMA function looks like this:

### EMA<sub>*t*</sub> = α ∗ (*t*<sub>0</sub> − EMA<sub>*t−1*</sub>) + EMA<sub>*t−1*</sub>

Where:

- t<sub>0</sub> = Latest value
- t<sub>−1</sub> = Previous value
- EMA<sub>*t−1*</sub> = Exponential moving average of previous day.
- The multiplier: α = 2/(M+1)

and 

- M = Number of time periods, span of the window

However, we will use the pandas `ewm()` (Exponential Weighted functions) to compute our EMA. So we just need to define the following:

M = `span` argument = number of time periods. We will try 7 days, 14 days, and 30 days.

## Bollinger Bands

In [ ]:
# Data + Prep from TS: explore lesson
plt.rc('figure', figsize=(13, 6))
plt.rc('axes.spines', top=False, right=False)
plt.rc('font', size=13)

# T(°F) = T(K) × 9/5 - 459.67
def to_fahrenheit(k):
    return k * 9/5 - 459.67

url = "https://gist.githubusercontent.com/ryanorsinger/0ec766c66f4089bdcbc1d4fb294a3394/raw/197c1f0d7b55a45f29437811bc73d9c4ef8af647/sa_temps.csv"
s = pd.read_csv(url, index_col='datetime', parse_dates=True).temp
s = s.dropna()
s = to_fahrenheit(s)
s = s.resample('D').mean()

## Bollinger Bands

- A volatility indicator commonly used in fiscal and trading analysis.
- Made up of 3 lines, the Upper Band (UB), the Lower Band (LB) and the Midband.
- Based on the exponential moving average

### Midband

The Exponential Moving Average

`midband = train.ewm(span=30).mean()`

### Upper & Lower Band

UB/LB = Midband +/- stdev * K

`stdev = train.ewm(span=30).std()`

K = the number of standard deviations to go up and down from the EMA

### %b, Percent Bandwidth

Shows where the last value sits in relation to the bands.

%b = last−LB / UB-LB

- If %b > 1, data point lies above the upper band
- If %b < 0, data point lies below the lower band
- If %b == .5, data point lies on the midband.

In [ ]:
K = 
M = 
# std = s.rolling(M).std()
std = 
bands = 
# bands['mid'] = s.rolling(M).mean()
bands['mid'] = 
bands['upper'] = 
bands['lower'] = 
bands['actual'] = 

t = bands.loc['2013']
t[['upper', 'lower']].plot(color='black', alpha=.6, ls=':', figsize=(16, 6))
t.mid.plot(color='black', alpha=.6, ls='--')
t.actual.plot()
plt.legend('')
plt.xlabel('')

In [ ]:
bands['%b'] = 

In [ ]:
upper_outliers = 

In [ ]:
lower_outliers = 

In [ ]:
plt.plot(bands.index, bands.actual, label='Temperature (deg F)')
plt.vlines(upper_outliers.index, *plt.ylim(), color='black', ls='--', label='Upper Outlier')
plt.title('San Antonio Temperature Over Time')
plt.legend()

In [ ]:
plt.plot(bands.index, bands.actual, label='Temperature (deg F)')
plt.vlines(lower_outliers.index, *plt.ylim(), color='black', ls='--', label='Lower Outlier')
plt.title('San Antonio Temperature Over Time')
plt.legend()

In [ ]:
plt.plot(bands.index, bands.actual, label='Temperature (deg F)')
plt.vlines(upper_outliers.index, *plt.ylim(), color='black', ls='--', label='Upper Outlier')
plt.vlines(lower_outliers.index, *plt.ylim(), color='black', ls=':', label='Lower Outlier')
plt.title('San Antonio Temperature Over Time')
plt.legend()
plt.xlim(pd.to_datetime('2013'), pd.to_datetime('2014'))

How do we deal with "dense" data? Use less data points:

- consider a broader resampling period
- use a subset of the data ("zoom in")
- change the threshold for outlier detection -- in this case, use a higher K value
- experiment and iterate

"If you torture the data enough it will confess to anything"

- Is this analysis reproducible with new data?
- If my worst enemy read this work, what would they criticize?
- State your hypothesis, biases, and assumptions ahead of time

## So how do we apply this to the project we are working on?

- data that is not uniformly frequent
- data that has lots of discrete features

A: We create a continuous variable from the discrete ones by resampling + .size().

In [ ]:
import env

url = f'mysql+pymysql://{env.user}:{env.password}@{env.host}/curriculum_logs'
query = 'SELECT * FROM logs'

df = pd.read_sql(query, url)

In [ ]:
df.head()

I'm choosing to focus on cohort_id

In [ ]:
# subset
df = df[df.cohort_id.isin([33, 34, 51])]

In [ ]:
df = df.set_index(pd.to_datetime(df.date + ' ' + df.time))

This is the **secret sauce**

1. groupby + resample + `.size()` gives us counts over time for each unique value of the discrete variable
1. `.unstack(0)` converts the index of the discrete variable to columns

We end up with a dataframe where the rows are our chosen time period, columns are unique values of the discrete var, and the values are the number of times that combination of discrete var + time period shows up (i.e. # of requests per cohort per week).

In [ ]:
cohort_requests = 


In [ ]:
K = 
M = 
s = 
bands = 
std = 
# std = s.rolling(M).std()
bands['actual'] = 

# bands['mid'] = bands.actual.rolling(M).mean()
bands['mid'] = 
bands['upper'] = 
bands['lower'] = 
bands.head()

In [ ]:
bands[['upper', 'lower']].plot(color='black', alpha=.6, ls=':', figsize=(16, 6))
bands.mid.plot(color='black', alpha=.6, ls='--')
bands.actual.plot()
plt.legend('')
plt.xlabel('')
plt.title('weekly # of requests for cohort_id=33')
plt.ylabel('# of requests')

Next steps:

1. Calculate %b to call out outliers and visualize
1. tweak hyperparams -- try a ewma, diff values for k
1. apply this to multiple cohorts or other discrete variables (a function might help!)